# 1. Code from the Network file to define the countries

In [9]:
import requests
import json
import re
import networkx as nx
import numpy as np
import os
from io import StringIO
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from collections import Counter
import nltk
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rital\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
baseurl = "https://en.wikipedia.org/w/api.php?"
action = "action=query"
title = "titles=Member_states_of_the_United_Nations"
content = "prop=revisions&rvprop=content"
dataformat ="format=json"

query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
print(query)

https://en.wikipedia.org/w/api.php?action=query&prop=revisions&rvprop=content&titles=Member_states_of_the_United_Nations&format=json


In [5]:
headers = {"User-Agent" : "MyWikipediaClient/1.0 (example@example.com)"} # just use this dict as-is.
wikiresponse = requests.get(query, headers=headers)
wikidata = wikiresponse.text
dict_data=json.loads(wikidata)

In [6]:
raw_data=dict_data['query']['pages']['31969']['revisions'][0]['*']
raw_data

'{{Short description|none}}\n{{For|the League of Nations|Member states of the League of Nations}}\n{{Featured list}}\n{{pp-pc|small=yes}}\n{{Use dmy dates|date=February 2025}}\n{{Use Oxford spelling|date = February 2025}}\n\n[[File:United Nations (Member States and Territories).svg|thumb|\n{{legend|#009edb|193 United Nations member states}}\n{{legend|#9edb00|2 [[United Nations General Assembly observers#Current non-member observers|UN General Assembly observer states]] (the [[Holy See]]{{efn|The [[sovereignty|sovereign entity]] with [[state (polity)|statehood]] over the territory of the [[Vatican City|Vatican City State]].}} and [[Palestine]])}}\n{{legend|#db9e00|2 non-member states (the [[Cook Islands]] and [[Niue]])<ref>{{Cite web|url = https://www.un.org/en/development/desa/population/publications/pdf/policy/WPP2013/Chapters/a_Preface.pdf |title = UN: "Data for 197 countries: all 193 member states, two observer states (the Holy See and Palestine), and two non-member states (Cook Isl

In [7]:
countries = re.findall(r"\{flag(?:deco|country)?\|([^|}]+)", raw_data)
no_countries = len(countries)
print("Number of countries:", no_countries)

Number of countries: 193


# 2. Reddit API acess

### Test for Portugal to see if its working

In [11]:

url = "https://www.reddit.com/r/history/search.json"
params = {
    "q": "Portugal",
    "limit": 100,
    "sort": "relevance",
    "t": "all",
    "restrict_sr": 1
}

headers = {"User-agent": "Mozilla/5.0"}

response = requests.get(url, params=params, headers=headers)
print(response.status_code)
print(response.text[:500])
data = response.json()

posts = data["data"]["children"]

#print(len(posts))
#print(posts[0]["data"]["title"])


429
<!doctype html>
<html>
  <head>
    <title>Too Many Requests</title>
    <style>
      body {
          font: small verdana, arial, helvetica, sans-serif;
          width: 600px;
          margin: 0 auto;
      }

      h1 {
          height: 40px;
          background: transparent url(//www.redditstatic.com/reddit.com.header.png) no-repeat scroll top right;
      }

      textarea.mushroom {
          display: none
      }
    </style>
  </head>
  <body>
    <h1>whoa there, pardner!</h1>

<p>Re


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
print(posts[0]["data"].keys())

NameError: name 'posts' is not defined

In [27]:
print(len(posts))

100


In [34]:
print(posts[75]["data"]["selftext"])

The Scramble for Africa only really began in the 1880s.

Sure, there were small exceptions:

 France has outposts in Senegal since the 17th century, Islands (Reunion), Algerian Conquest began in 1830...

Both the Dutch and British started colonizing South Africa earlier. 

Portugal seems to be the only country who began their colonization of Angola and Mozambique quite early (16th century). 

Why did the European Powers (bar Portugal)  get most of their African colonies so late?

Why did they colonise North America, India and Asia so much earlier than Africa, a continent that was right under them!

This especially goes for France (who ended up getting one of the largest chunks): after it lost its colonial empire to Britain at the end of the 7 year war, wouldn't it have been eager to regain it's prestige and rebuild an empire?

Why was most of Africa colonized so late?


### Function to retrieve only posts that are non empty

In [35]:
import requests
import time

headers = {"User-agent": "Mozilla/5.0"}

def fetch_posts(keyword, subreddit=None, limit=100, sort="relevance", t="all"):
    """
    Fetch up to 100 raw Reddit posts for a given keyword/subreddit.
    Does NOT filter empty posts.
    """
    if subreddit:
        url = f"https://www.reddit.com/r/{subreddit}/search.json"
        params = {
            "q": keyword,
            "limit": limit,
            "sort": sort,
            "t": t,
            "restrict_sr": 1
        }
    else:
        url = "https://www.reddit.com/search.json"
        params = {
            "q": keyword,
            "limit": limit,
            "sort": sort,
            "t": t
        }

    try:
        response = requests.get(url, params=params, headers=headers)
        data = response.json()
        return data["data"]["children"]
    except:
        return []


def is_valid_post(post):
    """A post is valid if it has title or selftext not empty."""
    data = post["data"]
    title = data.get("title", "").strip()
    selftext = data.get("selftext", "").strip()

    # Reject ads, promoted posts, empty posts
    if data.get("is_self") is False:
        return False
    
    return (title != "" or selftext != "")


def get_100_valid_posts(keyword, subreddit=None):
    """
    Returns exactly 100 valid posts for a given country query.
    Fetches multiple times until 100 valid posts are collected.
    """
    valid_posts = []
    attempts = 0

    # Multiple strategies to get enough posts
    time_windows = ["all", "year", "month", "week", "day"]
    sorts = ["relevance", "new", "top"]

    while len(valid_posts) < 100:
        attempts += 1
        
        t = time_windows[attempts % len(time_windows)]
        sort = sorts[attempts % len(sorts)]

        print(f"🔎 Attempt {attempts}: sort={sort}, time={t}")

        raw = fetch_posts(keyword, subreddit=subreddit, limit=100, sort=sort, t=t)

        for post in raw:
            if is_valid_post(post):
                valid_posts.append(post)

            if len(valid_posts) == 100:
                break

        # Avoid hammering Reddit
        time.sleep(0.5)

        # Safety check (avoid infinite loops)
        if attempts > 15:
            print("⚠️ Warning: Could not get 100 valid posts. Returning what was found.")
            break

    return valid_posts[:100]


In [36]:
posts = get_100_valid_posts("Portugal", "history")


🔎 Attempt 1: sort=new, time=year
🔎 Attempt 2: sort=top, time=month
🔎 Attempt 3: sort=relevance, time=week
🔎 Attempt 4: sort=new, time=day
🔎 Attempt 5: sort=top, time=all
🔎 Attempt 6: sort=relevance, time=year
🔎 Attempt 7: sort=new, time=month
🔎 Attempt 8: sort=top, time=week
🔎 Attempt 9: sort=relevance, time=day
🔎 Attempt 10: sort=new, time=all


In [38]:
print(len(posts))

100


In [42]:
print(posts[2]["data"]["selftext"])

I thought you guys might like some of the historical illustrations that can be found on /r/papertowns. The countries and cities are listed alphabetically in one of the 5 categories: Europe, Asia &amp; Oceania, The Americas, Middle East &amp; Egypt, Africa. Countries such as Russia, Turkey, Cyprus, Armenia and Georgia are found in the Europe section.

Hope you'll enjoy!

___

**EUROPE**
___

Albania:

* [Buthrotum, antiquity, modern Butrint](https://i.imgur.com/AHywz2Q.jpg)

Armenia:

* [Yerevan, 1672](https://upload.wikimedia.org/wikipedia/commons/f/f6/05_Chardin_Yerevan1672.gif)

Austria:

* [Iuvavum, antiquity, modern Salzburg](https://i.imgur.com/twJsVhG.jpg)

* [Schwaz, 16th century](https://i.imgur.com/JUIDHCl.jpg)

* [Vienna, 1640](https://upload.wikimedia.org/wikipedia/commons/c/ca/Wien_1609_1640_ArM.jpg)

* [Vienna, 1683](https://upload.wikimedia.org/wikipedia/commons/1/1e/Wien-1683%281686%29-Allen-uncroped.jpg)

Belarus:

* [Brest, 17th century](https://i.redd.it/u3y7h1yuuirx.

# 3. Apply this function to our countries

In [43]:
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 "Côte d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'North Korea',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'The Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea Bissau',
 'Guyana',
 'Haiti'

In [ ]:
countries_posts = {}   # FINAL structure you want

for country in countries:
    print(f"🔎 Getting posts for: {country}")

    posts = get_100_valid_posts(country, subreddit="history")
    
    # extract ONLY the selftexts
    selftexts = []
    for p in posts:
        t = p["data"].get("selftext", "").strip()
        if t != "":
            selftexts.append(t)

    countries_posts[country] = selftexts
    print(f"   → {len(selftexts)} valid selftexts\n")


🔎 Getting posts for: Afghanistan
🔎 Attempt 1: sort=new, time=year
🔎 Attempt 2: sort=top, time=month
🔎 Attempt 3: sort=relevance, time=week
🔎 Attempt 4: sort=new, time=day
🔎 Attempt 5: sort=top, time=all
🔎 Attempt 6: sort=relevance, time=year
🔎 Attempt 7: sort=new, time=month
🔎 Attempt 8: sort=top, time=week
🔎 Attempt 9: sort=relevance, time=day
🔎 Attempt 10: sort=new, time=all
   → 100 valid selftexts

🔎 Getting posts for: Albania
🔎 Attempt 1: sort=new, time=year
🔎 Attempt 2: sort=top, time=month
🔎 Attempt 3: sort=relevance, time=week
🔎 Attempt 4: sort=new, time=day
🔎 Attempt 5: sort=top, time=all
🔎 Attempt 6: sort=relevance, time=year
🔎 Attempt 7: sort=new, time=month
🔎 Attempt 8: sort=top, time=week
🔎 Attempt 9: sort=relevance, time=day
🔎 Attempt 10: sort=new, time=all
🔎 Attempt 11: sort=top, time=year
🔎 Attempt 12: sort=relevance, time=month
🔎 Attempt 13: sort=new, time=week
🔎 Attempt 14: sort=top, time=day
🔎 Attempt 15: sort=relevance, time=all
🔎 Attempt 16: sort=new, time=year
⚠️ 

# 4. Sentiment Analysis 